In [19]:
import os
import numpy as np
import re
import keras
import tensorflow as tf
from keras import *
from keras.preprocessing.text import *
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
from gensim.models import word2vec
from keras.preprocessing.text import Tokenizer

A function from set 6 to parse the observations:

In [2]:
def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [133]:
text = open(os.path.join(os.getcwd(), \
                         'Release/data/shakespeare.txt')).read()
obs, obs_map = parse_observations(text)
all_words = open(os.path.join(os.getcwd(), \
                         'Release/data/shakespeare.txt')).read()

In [223]:
all_chars = list(text)
all_words = text.split()
words = sorted(set(all_words))
# print(len(words))
# words2 = sorted(list(set(text.split())))
# print(len(words2))
chars = sorted(list(set(text)))
data_size, vocab_size = len(text), len(chars)
word_to_index = {char:idx for idx, char in enumerate(words)}
index_to_word = {idx:char for idx, char in enumerate(words)}

In [224]:
n_step = 2
s_len = 10
training_data = []
next_word = []
for i in range(0, len(all_words)-s_len, n_step):
    
    string_to_add = all_words[i:i + s_len]
    training_data.append(string_to_add)
    if i + s_len < len(all_words)-1:
        next_word.append(all_words[i + s_len])
    
# This is our end symbol: a space.
#next_word.append('a')

In [225]:
print(len(training_data))
print(len(next_word))

8863
8863


In [226]:
trainX = np.zeros((len(training_data), s_len, len(words)))
trainY = np.zeros((len(training_data), len(words)))

for sent_idx in range(0, len(training_data)):
    curr_sentence = training_data[sent_idx]
    curr_next_word = next_word[sent_idx]
    for i, word in enumerate(curr_sentence):
        trainX[sent_idx, i, word_to_index[word]] = 1
    trainY[sent_idx, word_to_index[curr_next_word]] = 1

In [227]:
list_train_data = []
for data in training_data:
    list_train_data.append(list(data))

In [208]:
print(len(training_data))
print(token_model_seqs[0][0])

49010
0.0


In [166]:
tokenizer = Tokenizer()
#text_tokens = tokenizer.texts_to_sequences(texts = list(training_data))
tokenizer.fit_on_texts(list(training_data))
token_model_seqs = tokenizer.texts_to_matrix(training_data)
print(token_model_seqs)


X = token_model_seqs.reshape(1, len(token_model_seqs), len(token_model_seqs[0]))
y = tokenizer.texts_to_matrix(next_word)

print(X.shape)
# for sent_idx in range(0, len(training_data)):
#     curr_sentence = training_data[sent_idx]
#     curr_next_word = next_word[sent_idx]
#     for i, word in enumerate(curr_sentence):
#         trainX[sent_idx, i, word_to_index[word]] = 1
#     trainY[sent_idx, word_to_index[curr_next_word]] = 1

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(1, 49010, 10727)


In [47]:
X = token_model_seqs
y = tokenizer.texts_to_matrix(next_word)

In [48]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

3350


In [279]:
def print_poem_maxprob(length, random = False):
    if random == True:
        
        start_index = np.random.randint(0, len(text.split())-s_len-1)
        sentence1 = np.array(all_words[start_index: start_index + s_len])
        
    else:
        sentence1 = np.array(['Shall', 'I', 'compare', 'thee', 'to', 'a', 'summer\'s', 'day', 'Thou', 'art'])
    #print("sentence", sentence)
    sentence = np.ndarray.tolist(sentence1)
    sequence = np.ndarray.tolist(sentence1)
    for i in range(length):
        x_pred = np.zeros((1, s_len, len(words)))
        for j in range(len(sentence)):
            x_pred[0, j, word_to_index[sentence[j]]] = 1.

        predictions = np.array(model.predict(x_pred)[0])
        max_index = np.argmax(predictions)
        next_word = index_to_word[max_index]
        #print(next_word)
        sentence.append(next_word)
        sentence = sentence[1:]
        sequence.append(next_word)
    
    #''.join(sequence)
    output = ' '.join(str(x) for x in sequence)
    print(output)
    
def print_poem_randprob(length, random = False):
    if random == True:
        
        start_index = np.random.randint(0, len(text.split())-s_len-1)
        sentence1 = np.array(all_words[start_index: start_index + s_len])
        
    else:
        sentence1 = np.array(['Shall', 'I', 'compare', 'thee', 'to', 'a', 'summer\'s', 'day', 'Thou', 'art'])
        
    sentence = np.ndarray.tolist(sentence1)
    sequence = np.ndarray.tolist(sentence1)
    for i in range(length):
        x_pred = np.zeros((1, s_len, len(words)))
        for j in range(len(sentence)):
            x_pred[0, j, word_to_index[sentence[j]]] = 1.

        predictions = np.array(model.predict(x_pred)[0])
        index = np.random.choice(len(words), 1, p=predictions)[0]
        max_index = np.argmax(predictions)
        next_word = index_to_word[max_index]
        #print(next_word)
        sentence.append(next_word)
        sentence = sentence[1:]
        sequence.append(next_word)
    
    #''.join(sequence)
    output = ' '.join(str(x) for x in sequence)
    print(output)
    

In [281]:
model = Sequential()
model.add(LSTM(128, input_shape=(s_len, len(words))))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

#train the model

#print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop')
model.fit(trainX, trainY, batch_size = 128, nb_epoch = 100)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/100
8863/8863 [==============================] - 74s 8ms/step - loss: 7.3512
Epoch 2/100
8863/8863 [==============================] - 64s 7ms/step - loss: 6.8719
Epoch 3/100
8863/8863 [==============================] - 64s 7ms/step - loss: 6.7830
Epoch 4/100
8863/8863 [==============================] - 62s 7ms/step - loss: 6.7201
Epoch 5/100
8863/8863 [==============================] - 61s 7ms/step - loss: 6.6787
Epoch 6/100
8863/8863 [==============================] - 61s 7ms/step - loss: 6.6429
Epoch 7/100
8863/8863 [==============================] - 61s 7ms/step - loss: 6.6219
Epoch 8/100
8863/8863 [==============================] - 61s 7ms/step - loss: 6.5934
Epoch 9/100
8863/8863 [==============================] - 63s 7ms/step - loss: 6.5648
Epoch 10/100
8863/8863 [==============================] - 61s 7ms/step - loss: 6.5368
Epoch 11/100
8863/8863 [==============================] - 63s 7ms/step - loss: 6.5068
Epoch 12/100
8863/8863 [==============================] - 65s 7

8863/8863 [==============================] - 41s 5ms/step - loss: 1.2076
Epoch 97/100
8863/8863 [==============================] - 41s 5ms/step - loss: 1.1855
Epoch 98/100
8863/8863 [==============================] - 41s 5ms/step - loss: 1.1690
Epoch 99/100
8863/8863 [==============================] - 41s 5ms/step - loss: 1.1590
Epoch 100/100
8863/8863 [==============================] - 41s 5ms/step - loss: 1.1364


In [280]:
print_poem_maxprob(100, False)
print("end\n")

print_poem_randprob(100, False)

Shall I compare thee to a summer's day Thou art more more and my with too with This dost dost thy O it from So When So is yet much had The Not can so so numbers things hold do from a to to That to fair for As the left him him losing seeming his beauty beauty beauty have have he those have in with you you with am though though him Time bars wasteful change love's him change change he true their day let I are than hast the I to in mourn should loss blame me my in in I which I better in your mine with
end

Shall I compare thee to a summer's day Thou art more more and my with too with This dost dost thy O it from So When So is yet much had The Not can so so numbers things hold do from a to to That to fair for As the left him him losing seeming his beauty beauty beauty have have he those have in with you you with am though though him Time bars wasteful change love's him change change he true their day let I are than hast the I to in mourn should loss blame me my in in I which I better in yo